In [254]:
import numpy as np
import pandas as pd
train_df = pd.read_csv('/kaggle/input/train.csv',index_col=0)
submission_df = pd.read_csv('/kaggle/input/sample_submission.csv',index_col=0)
testFeaturesData = pd.read_csv('/kaggle/input/test.csv',index_col=0)
train_df = train_df.dropna()

New features are being generated using existing features.
* 'distance' (Generated using  pickup Lat/Lon and Dropoff Lat/Lon)
       Haversine distance was calculated using two lat/lons.
* 'phour', 'pday', 'pmonth', 'pyear', 'dhour', 'dday', 'dmonth', 'dyear' 
       These features were derived from 'pickup_time' and 'drop_time'


In [255]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

train_df['distance'] = haversine_array(train_df['pick_lat'],train_df['pick_lon'], train_df['pick_lat'], train_df['drop_lon']) + haversine_array(train_df['pick_lat'], train_df['pick_lon'], train_df['drop_lat'], train_df['pick_lon'])
testFeaturesData['distance'] = haversine_array(testFeaturesData['pick_lat'],testFeaturesData['pick_lon'], testFeaturesData['pick_lat'], testFeaturesData['drop_lon']) + haversine_array(testFeaturesData['pick_lat'], testFeaturesData['pick_lon'], testFeaturesData['drop_lat'], testFeaturesData['pick_lon'])
train_df = train_df.drop(columns=['pick_lat','pick_lon','drop_lat','drop_lon'])
testFeaturesData = testFeaturesData.drop(columns=['pick_lat','pick_lon','drop_lat','drop_lon'])

In [256]:
train_df.describe()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,distance
count,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000,16968.000000
mean,13.721437,1702.836987,628.875000,32.045122,112.489510,312.657884,6.210519
std,98.992493,8121.556167,8064.657846,465.564240,442.129943,606.372103,64.811056
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.500000,548.000000,33.000000,0.000000,26.000000,125.500000,1.870612
50%,10.500000,1009.000000,123.000000,0.115600,73.000000,211.050000,4.043496
75%,10.500000,1802.250000,351.000000,14.334400,144.000000,366.205000,7.766681
max,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,26945.990000,8413.972026


Feature dropping 
* 'drop_time' and 'pickup_time' were dropped after generating new features using those. 
* 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon' features  were dropped after generating 'distance'.


In [257]:
a = train_df['pickup_time'].str.slice(0, 15)
a= pd.to_datetime(a, utc=True, format='%m/%d/%Y %H:%M')
a.dt.hour
train_df['phour'] = a.dt.hour
train_df['pday'] = a.dt.day
train_df['pmonth'] = a.dt.month
train_df['pyear'] = a.dt.year

b = testFeaturesData['pickup_time'].str.slice(0, 15)
b= pd.to_datetime(b, utc=True, format='%m/%d/%Y %H:%M')
b.dt.hour
testFeaturesData['phour'] = b.dt.hour
testFeaturesData['pday'] = b.dt.day
testFeaturesData['pmonth'] = b.dt.month
testFeaturesData['pyear'] = b.dt.year

c = train_df['drop_time'].str.slice(0, 15)
c= pd.to_datetime(c, utc=True, format='%m/%d/%Y %H:%M')
c.dt.hour
train_df['dhour'] = c.dt.hour
train_df['dday'] = c.dt.day
train_df['dmonth'] = c.dt.month
train_df['dyear'] = c.dt.year

d = testFeaturesData['drop_time'].str.slice(0, 15)
d= pd.to_datetime(d, utc=True, format='%m/%d/%Y %H:%M')
d.dt.hour
testFeaturesData['dhour'] = d.dt.hour
testFeaturesData['dday'] = d.dt.day
testFeaturesData['dmonth'] = d.dt.month
testFeaturesData['dyear'] = d.dt.year
# train_df['timeDiff']=(pd.to_datetime(train_df['drop_time'], utc=True, format='%m/%d/%Y %H:%M')-pd.to_datetime(train_df['pickup_time'], utc=True, format='%m/%d/%Y %H:%M')).dt.seconds / 60.0
# testFeaturesData['timeDiff']=(pd.to_datetime(testFeaturesData['drop_time'], utc=True, format='%m/%d/%Y %H:%M')-pd.to_datetime(testFeaturesData['pickup_time'], utc=True, format='%m/%d/%Y %H:%M')).dt.seconds /60.0
train_df = train_df.drop('pickup_time', axis=1)
testFeaturesData = testFeaturesData.drop('pickup_time', axis=1)
train_df = train_df.drop('drop_time', axis=1)
testFeaturesData = testFeaturesData.drop('drop_time', axis=1)


In [258]:
trainLabelsData = train_df.filter(['label'], axis=1)
trainFeaturesData =  train_df.drop('label', axis=1)

In [259]:
label_classes = {'correct': 1,
                'incorrect': 0
                          }
trainLabelsData['label'] = trainLabelsData['label'].map(label_classes)

In [260]:
numerical_cols = np.array(['duration', 'meter_waiting','meter_waiting_fare', 'meter_waiting_till_pickup','fare','distance','dhour','dday','dmonth','dyear'])
boolean_cols = np.array(['additional_fare'])

In [261]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

Simple Imputer
* Mean imputation for boolean columns
* Median imputation for numerical columns

In [262]:
boolean_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='mean'))
])


numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numerical_cols),
        ("boolean", boolean_steps, boolean_cols)
    ],
    remainder = "drop"
)

In [263]:
X_train, X_eval, y_train, y_eval = train_test_split(trainFeaturesData, trainLabelsData, test_size=0.3, random_state=1) 

In [264]:
eval_set = [(X_eval, y_eval)]
estimator = XGBClassifier( max_depth=4, 
                          n_estimators=240, 
                          subsample=0.9,
#                           colsample_bytree=1, 
#                           colsample_bylevel=1, 
#                           colsample_bynode=1,
#                           gamma =0,
                          early_stopping_rounds=10,
                          eval_metric="logloss",
                          eval_set=eval_set)
# from catboost import CatBoostClassifier
# estimator = CatBoostClassifier(iterations=1500, 
#                            learning_rate=0.01, 
#                            l2_leaf_reg=3, 
#                            depth=9, 
#                            rsm=0.98, 
#                            loss_function= 
#                            'Logloss', 
#                            eval_metric='AUC',
#                            random_seed=42)

In [272]:
# import xgboost as xgb
# data_dmatrix = xgb.DMatrix(data=trainFeaturesData, label=trainLabelsData)
# params = {'objective':'reg:linear','colsample_bytree':0.3,'learning_rate':0.1,
#                 'max_depth':5, 'alpha':10}
# cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10, num_boost_round=50, early_stopping_rounds=10, metrics='rmse', as_pandas=True, seed=0) 

[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[19:41:08] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of re

In [277]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

In [278]:
results

array([0.95050088, 0.94401886, 0.92692988, 0.94519741, 0.96346494,
       0.95816146, 0.95521509, 0.96110784, 0.95283019, 0.93573113])

In [273]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimator),
])

In [274]:
full_pipeline.fit(X_train, y_train.values.ravel())
preds_ = full_pipeline.predict(X_eval)
preds = [round(value) for value in preds_]

In [275]:
y_preds = pd.DataFrame(
    {
        "label": preds
    },
    index = y_eval.index
)

In [276]:
f1_score(y_eval, y_preds, average='macro')

0.8268555803449964

In [269]:
# full_pipeline.fit(trainFeaturesData, trainLabelsData.values.ravel())
# results_ = full_pipeline.predict(testFeaturesData)
# results =[round(value) for value in results_]

In [270]:
# np.testing.assert_array_equal(testFeaturesData.index.values, 
#                               submission_df.index.values)
# submission_df["prediction"] = results
# submission_df.to_csv('my_submission.csv', index=True)